## Importing the CIFAR-100 dataset

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.preprocessing import LabelBinarizer

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='fine')

print(x_train.shape, y_train.shape)

(50000, 32, 32, 3) (50000, 1)


In [3]:
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False
[]


In [4]:
x_train = (x_train / 255 - 0.5) * 2
x_test = (x_test / 255 - 0.5) * 2

enc = LabelBinarizer()

y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)


In [5]:
print(x_test[0].shape)

(32, 32, 3)


## Inception v4 Implementation

In [15]:
#import tensorflow as tf
from keras.layers import Dense, Conv2D, Input, MaxPooling2D, BatchNormalization, Activation, AveragePooling2D, Dropout, Concatenate
#from keras.layers import *
from keras import Model

def MyInception():
    inputs = Input(shape=(32, 32, 3), name='input')

    x = Stem(inputs)

    for _ in range(4):
        x = InceptionA(x)

    x = ReductionA(x)

    for _ in range(7):
        x = InceptionB(x)

    x = ReductionB(x)

    for _ in range(3):
        x = InceptionC(x)

    x = AveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    outputs = Dense(100, activation='softmax', name='output')(x)
    model = Model(inputs=inputs, outputs=outputs, name='MyInception')

    return model

# TODO: Go to Inceptionv2 paper to understand Conv2D_BN and fine tune parameters potentially
def Conv2DBatchNorm(x, filters, kernel_size=(3, 3), strides=(1, 1), padding='same'):

    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
            #    kernel_regularizer=keras.regularizers.l2(0.00004),
            #    kernel_initializer=keras.initializers.VarianceScaling(scale=0.2, mode='fan_in', distribution='normal', seed=None)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

def Stem(x):
    x = Conv2DBatchNorm(x, 32, (3, 3), strides=(2, 2), padding='valid')
    x = Conv2DBatchNorm(x, 32, (3, 3), padding='valid')
    x = Conv2DBatchNorm(x, 64, (3, 3))

    branch_0 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)
    branch_1 = Conv2DBatchNorm(x, 96, (3, 3), strides=(2, 2), padding='valid')

    x = Concatenate()([branch_0, branch_1])

    branch_0 = Conv2DBatchNorm(x, 64, (1, 1))
    branch_0 = Conv2DBatchNorm(branch_0, 96, (3, 3), padding='valid')

    branch_1 = Conv2DBatchNorm(x, 64, (1, 1))
    branch_1 = Conv2DBatchNorm(branch_1, 64, (7, 1))
    branch_1 = Conv2DBatchNorm(branch_1, 64, (1, 7))
    branch_1 = Conv2DBatchNorm(branch_1, 96, (3, 3), padding='valid')

    x = Concatenate()([branch_0, branch_1])

    return x


def InceptionA(x):
    branch_0 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_0 = Conv2DBatchNorm(branch_0, 96, (1, 1))

    branch_1 = Conv2DBatchNorm(x, 96, (1, 1))

    branch_2 = Conv2DBatchNorm(x, 64, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 96, (3, 3))

    branch_3 = Conv2DBatchNorm(x, 64, (1, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 96, (3, 3))
    branch_3 = Conv2DBatchNorm(branch_3, 96, (3, 3))

    x = Concatenate()([branch_0, branch_1, branch_2, branch_3])

    return x


def InceptionB(x):
    branch_0 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_0 = Conv2DBatchNorm(branch_0, 128, (1, 1))

    branch_1 = Conv2DBatchNorm(x, 384, (1, 1))

    branch_2 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 224, (1, 7))
    branch_2 = Conv2DBatchNorm(branch_2, 256, (7, 1))

    branch_3 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 192, (1, 7))
    branch_3 = Conv2DBatchNorm(branch_3, 224, (7, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 224, (1, 7))
    branch_3 = Conv2DBatchNorm(branch_3, 256, (7, 1))

    x = Concatenate()([branch_0, branch_1, branch_2, branch_3])

    return x


def InceptionC(x):
    branch_0 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_0 = Conv2DBatchNorm(branch_0, 256, (1, 1))

    branch_1 = Conv2DBatchNorm(x, 256, (1, 1))

    branch_2 = Conv2DBatchNorm(x, 384, (1, 1))
    branch_2_0 = Conv2DBatchNorm(branch_2, 256, (1, 3))
    branch_2_1 = Conv2DBatchNorm(branch_2, 256, (3, 1))

    branch_3 = Conv2DBatchNorm(x, 384, (1, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 448, (1, 3))
    branch_3 = Conv2DBatchNorm(branch_3, 512, (3, 1))
    branch_3_0 = Conv2DBatchNorm(branch_3, 256, (3, 1))
    branch_3_1 = Conv2DBatchNorm(branch_3, 256, (1, 3))

    x = Concatenate()([branch_0, branch_1, branch_2_0, branch_2_1, branch_3_0, branch_3_1])

    return x


def ReductionA(x):
    branch_0 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    branch_1 = Conv2DBatchNorm(x, 384, (3, 3), strides=(2, 2), padding='valid')

    branch_2 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 224, (3, 3))
    branch_2 = Conv2DBatchNorm(branch_2, 256, (3, 3), strides=(2, 2), padding='valid')

    x = Concatenate()([branch_0, branch_1, branch_2])

    return x


def ReductionB(x):
    print(x.shape)
    branch_0 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    branch_1 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_1 = Conv2DBatchNorm(branch_1, 192, (3, 3), strides=(2, 2), padding='valid')

    branch_2 = Conv2DBatchNorm(x, 256, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 256, (1, 7))
    branch_2 = Conv2DBatchNorm(branch_2, 320, (7, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 320, (3, 3), strides=(2, 2), padding='valid')

    x = Concatenate()([branch_0, branch_1, branch_2])
    return x


model = MyInception()

model.compile()

model.summary()

(None, 1, 1, 1024)


ValueError: Exception encountered when calling layer "max_pooling2d_26" (type MaxPooling2D).

Negative dimension size caused by subtracting 3 from 1 for '{{node max_pooling2d_26/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 3, 3, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,1,1,1024].

Call arguments received by layer "max_pooling2d_26" (type MaxPooling2D):
  • inputs=tf.Tensor(shape=(None, 1, 1, 1024), dtype=float32)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 1, 1, 96), (None, 4, 4, 96), (None, 4, 4, 96), (None, 4, 4, 96)]